<a href="https://colab.research.google.com/github/leejineun/ai-class/blob/main/7_covid_xray_%EC%88%98%EC%97%85%EC%9A%A9(0819).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

In [ ]:
!ls

In [ ]:
!unzip coronahack-chest-xraydataset.zip

In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D #GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, MaxPool2D #BatchNormalization, AvgPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam

import tensorflow as tf

import os
import numpy as np
import pandas as pd
from os import listdir

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
metadata_path = './Chest_xray_Corona_Metadata.csv'
metadata = pd.read_csv(metadata_path, index_col=0)

In [ ]:
metadata.head()

In [ ]:
metadata['Label'].unique()

In [ ]:
def check_file_name(file_name) : 
    if 'virus' in file_name : 
        return True
    elif 'bacteria' in file_name : 
        return True
    else: 
        print(file_name)
        return False

In [ ]:
metadata[metadata['Label'] == 'Pnemonia']['X_ray_image_name'].apply(lambda x : check_file_name(x)).unique()

In [ ]:
metadata['Label_2_Virus_category'].unique()

In [ ]:
train_set_dir = './Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
test_set_dir = './Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'

In [ ]:
train_df = pd.DataFrame(listdir(train_set_dir), columns=['path'])
train_df.head()

In [ ]:
train_df = pd.merge(train_df, metadata, left_on='path', right_on='X_ray_image_name', )

In [ ]:
train_df['path'] = train_df['path'].apply(lambda x : f'{train_set_dir}/{x}')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df = pd.DataFrame(listdir(test_set_dir), columns=['path'])
test_df.head()

In [ ]:
test_df = pd.merge(test_df, metadata, left_on='path', right_on='X_ray_image_name', )

In [ ]:
test_df['path'] = test_df['path'].apply(lambda x : f'{test_set_dir}/{x}')
test_df.head()

In [ ]:
test_df.shape

In [ ]:
train_df['path'].unique()

In [ ]:
import matplotlib.pyplot as pltb
import matplotlib.image as mpimg

In [ ]:
image_from_path = mpimg.imread(train_df['path'][0])
image_from_path.shape

In [ ]:
train_df['path'][train_df['Label'] == 'Normal']

In [ ]:
plt.figure(figsize=(12,12))
for i in range(16) : 
    plt.subplot(4,4, i+1)
    if i < 8 : 
        image_from_path = mpimg.imread(train_df['path'][train_df['Label'] == 'Normal'].iloc[i])
        plt.title('Normal')
        plt.imshow(image_from_path, cmap='bone')
    else : 
        image_from_path = mpimg.imread(train_df['path'][train_df['Label'] == 'Pnemonia'].iloc[i])
        plt.title('Pnemonia')
        plt.imshow(image_from_path, cmap='bone')
plt.show()

In [ ]:
import seaborn as sns
sns.countplot(train_df['Label'])
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
label_encoder.fit(train_df['Label'])
train_df['Label'] = label_encoder.transform(train_df['Label'])
train_df['Label'].head()

In [ ]:
test_df['Label'] = label_encoder.fit_transform(test_df['Label'])
test_df['Label'].head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
generator = ImageDataGenerator(validation_split=0.2)

In [ ]:
train_df.iloc[0]

In [ ]:
train_df['Label'] = train_df['Label'].astype('string')
test_df['Label'] = test_df['Label'].astype('string')

In [ ]:
train_images = generator.flow_from_dataframe(dataframe=train_df, x_col='path', y_col='Label',class_mode='binary', subset='training', target_size=(128, 128), color_mode='grayscale')
validation_images = generator.flow_from_dataframe(dataframe=train_df, x_col='path', y_col='Label',class_mode='binary', subset='validation', target_size=(128, 128), color_mode='grayscale')

In [ ]:
test_images = generator.flow_from_dataframe(dataframe=test_df, x_col='path', y_col='Label',class_mode='binary', target_size=(128, 128), color_mode='grayscale')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3,3), activation='relu',))
model.add(Conv2D(250, (3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(256 ,(2,2), activation='relu',))
model.add(MaxPool2D(2,2))
    
model.add(Flatten())
model.add(Dense(32, activation='relu',))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
model_checkpoint = ModelCheckpoint('first_trial.h5', monitor='val_loss', save_best_only=True)
history = model.fit(train_images,
                    epochs=30,
                    validation_data=validation_images, 
                    callbacks=[early_stop, model_checkpoint]
                   )